# Cycle GAN

## Build Model

In [ ]:
import sys
sys.path.append('../')

from cycle_gan import CycleGAN
from data_loader.reside_data_loader import RESIDEOTSDataLoader as DataLoader
import numpy as np
from skimage.measure import compare_psnr, compare_ssim
import tensorflow as tf

model = CycleGAN()
dl = DataLoader()

## Train Model

In [ ]:
datasetX, datasetY = dl.train_data()
model.train(5, datasetX, datasetY, 8, 10000)

## Evaluate

In [ ]:
def evaluate(model, datasetX, datasetY, batch_size):

    psnr_list, ssim_list = [], []
    
    for image_x, image_y in tf.data.Dataset.zip((datasetX, datasetY)).batch(batch_size):
        fake_y = model.predict(image_x)
        for i in range(image_y.shape[0]):
            psnr_list.append(compare_psnr(image_y.numpy()[i], fake_y[i], data_range=1))
            ssim_list.append(compare_ssim(image_y.numpy()[i], fake_y[i], 3))

    return psnr_list, ssim_list

datasetX, datasetY = dl.test_data()
psnr_list, ssim_list = evaluate(model.generator_f, datasetX, datasetY, 8)

print(np.mean(psnr_list), np.mean(ssim_list))